In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S3'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #From 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 868
Total Tokens Found = 11157
Total Tokens Output = 10861
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 868
Total Tokens= 10861
Unique Tokens= 2908
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 868
Sorted Token Size = 2908
Clean Token Size = 983
*Stop Replacements here
References Processed =  868
Total Replacement Pairs = 7
Tokens Read = 10861
Tokens Changed =  9
References Changed = 9


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [6]:
entropy_F1(mu=0.5,muIncr=0.1,beta=9,sigma=95,epsilon=4.0,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 4.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1018
NewMatrix Iterateblocks Total Time = 0.6870285347104073


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 4.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 618
NewMatrix Iterateblocks Total Time = 0.5603481493890285


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 4.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 313
NewMatrix Iterateblocks Total Time = 0.4134409138932824


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 4.0
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 162
NewMatrix Iterateblocks Total Time = 0.2927958592772484


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 4.0
Running NewMatrix Jar!
 Single-Reference Blo

0.92727273

Create functions for variable transformation.

In [7]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [8]:
inv_softplus(1)

0.541324854612918

In [9]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [10]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [11]:
import os, sys

In [12]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [13]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [14]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [17]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-1.76332880889149

In [15]:
optimizer=None

In [16]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [17]:
from bayes_opt import SequentialDomainReductionTransformer

In [18]:
bounds_transformer = SequentialDomainReductionTransformer()

In [19]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [20]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(9), 
#             "sigma_sp": inv_softplus(95),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(4.0)},
#     lazy=True,
# )

In [21]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  1.396    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  1.759    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        |  0.9424   |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        | -1.438    |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        |  0.5878   |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  2.132    |  5.854    |  179.0    | -3.177    |  1.268    |  461.5    |
Mu:0.78|Mu+:0.04|Beta:5.00|Sigma:461.00|Epsilon:178.37|EpIter:0.00
|  40       |  2.132    |  5.767    |  178.4    | -3.179    |  1.265    |  461.4    |
Mu:0.77|Mu+:0.04|Beta:5.00|Sigma:460.00|Epsilon:178.55|EpIter:0.00
|  41       |  2.187    |  5.919    |  178.6    | -3.195    |  1.233    |  461.0    |
Mu:0.78|Mu+:0.04|Beta:6.00|Sigma:460.00|Epsilon:179.02|EpIter:0.00
|  42       |  1.802    |  6.083    |  179.0    | -3.181    |  1.26     |  460.9    |
Mu:0.77|Mu+:0.04|Beta:6.00|Sigma:461.00|Epsilon:179.33|EpIter:0.00
|  43       |  1.812    |  6.862    |  179.3    | -3.201    |  1.218    |  461.0    |
Mu:0.78|Mu+:0.04|Beta:6.00|Sigma:460.00|Epsilon:178.56|EpIter:0.00
|  44       |  1.802    |  6.593    |  178.6    | -3.183    |  1.256    |  461.0    |
Mu:0.77|Mu+:0.04|Beta:6.00|Sigma:461.00|Epsilon:178.71|EpIter:0.00
|  45       |  1.843    |  6.994    |  178.7    | -3.184    |  1.222    |  461.7  

In [22]:
optimizer.max

{'target': 2.187072219016914,
 'params': {'beta_sp': 5.822927472632371,
  'epsilon_sp': 178.60761614677568,
  'muIncr_logit': -3.1914920667146855,
  'mu_logit': 1.237959120694906,
  'sigma_sp': 461.6747802029938}}

In [23]:
inv_logit(optimizer.max["target"])

0.89908257

In [24]:
params=optimizer.max["params"]

In [25]:
softplus(params["beta_sp"])

5.82588203388037

In [26]:
softplus(params["sigma_sp"])

461.6747802029938

In [27]:
inv_logit(params["mu_logit"])

0.7752085699102462

In [28]:
inv_logit(params["muIncr_logit"])

0.03948714967111403

In [29]:
softplus(params["epsilon_sp"])

178.60761614677568

In [30]:
entropy_F1(mu=0.7752,
           muIncr=0.0395,
           beta=5,
           sigma=461,
           epsilon=178.607,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.7752
Starting Iteration epsilon= 178.607
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 834
NewMatrix Iterateblocks Total Time = 0.5898919720202684

Precision = TP / (TP + FP) ......................... = 0.91588785
Recall = TP / (TP + FN) ............................ = 0.88288288
F-Measure .......................................... = 0.89908257


0.89908257